<a href="https://colab.research.google.com/github/KPavanKumarV/Taking-to-Data-NL2SQL/blob/main/%5Bvpkr2004_gmail_com%5DL15GFG_ipyn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl "https://api.mockaroo.com/api/05c80580?count=1000&key=cd24d570" > "Employee_data.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  188k    0  188k    0     0  67466      0 --:--:--  0:00:02 --:--:-- 67449


In [ ]:
import sqlite3
import pandas as pd
import os
employees_schema = """
create table MOCK_DATA (
	employee_id INT,
	first_name VARCHAR(50),
	last_name VARCHAR(50),
	age INT,
	email VARCHAR(50),
	gender VARCHAR(50),
	job_title VARCHAR(50),
	department VARCHAR(9),
	salary DECIMAL(8,2),
	hire_date DATE,
	manager_id INT,
	street_address VARCHAR(50),
	city VARCHAR(50),
	state VARCHAR(50),
	postal_code VARCHAR(50),
	country VARCHAR(50),
	phone_number VARCHAR(50),
	emergency_contact VARCHAR(50),
	emergency_contact_relationship VARCHAR(7),
	start_time VARCHAR(50),
	end_time VARCHAR(50)
);
"""

In [ ]:
db_name = 'ecommerce.db'
if os.path.exists(db_name):
    os.remove(db_name)
    print(f"Removed existing database '{db_name}'.")

Removed existing database 'ecommerce.db'.


In [ ]:
import sqlite3
import pandas as pd
import os



COLUMN_DATA_TYPES = {
    'employees': {
        'employee_id': 'int64',
        'first_name': 'object',
        'last_name': 'object',
        'age' : 'int64',
        'email': 'object',
        'gender': 'object',
        'job_title': 'object',
        'department': 'object',
        'salary': 'float64',
        'hire_date' : 'datetime64',
        'manager_id': 'int64',
        'street_address': 'object',
        'city': 'object',
        'state': 'object',
        'postal_code': 'object',
        'country': 'object',
        'phone_number': 'object',
        'emergency_contact': 'object',
        'emergency_contact_relationship': 'object',
        'start_time': 'object',
        'end_time': 'object', # Added the missing closing quote here
    }
}

# --- Database setup ---
db_name = 'ecommerce.db'
conn = None  # Initialize connection to None

try:
    # Establish a connection to the SQLite database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    print(f"Database '{db_name}' created and connected successfully. ✅")

    # Create tables
    cursor.execute(employees_schema)

    print("Tables 'employees' created successfully.")

    # --- Load data from CSV files into the tables using pandas ---
    csv_to_table_map = {
        '/content/Employee_data.csv': 'employees',
    }

    for csv_file, table_name in csv_to_table_map.items():
        if os.path.exists(csv_file):
            print(f"\nProcessing '{csv_file}' for table '{table_name}'...")

            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(csv_file)

            # 1. Get the expected schema for the current table
            expected_schema = COLUMN_DATA_TYPES[table_name]
            expected_cols = list(expected_schema.keys())

            # 2. Handle missing/extra columns
            # Drop columns from DataFrame that are not in the schema
            df = df[df.columns.intersection(expected_cols)]

            # Add any missing columns and fill with None (which becomes NULL in SQL)
            for col in expected_cols:
                if col not in df.columns:
                    df[col] = None

            # 3. Reorder columns to match the defined schema exactly
            df = df[expected_cols]

            # 4. Enforce data types
            for col, dtype in expected_schema.items():
                if 'datetime' in dtype:
                    # Use pd.to_datetime for date/time columns, coercing errors to NaT (Not a Time)
                    df[col] = pd.to_datetime(df[col], errors='coerce')
                else:
                    # Use astype for other columns, handling potential conversion errors
                    try:
                        df[col] = df[col].astype(dtype)
                    except (ValueError, TypeError) as e:
                        print(f"  - Warning: Could not convert column '{col}' to {dtype}. Error: {e}. Leaving as is.")


            # Use the to_sql method to insert the cleaned DataFrame
            df.to_sql(table_name, conn, if_exists='append', index=False)
            print(f"  -> Data from '{csv_file}' loaded into '{table_name}' table successfully.")
        else:
            print(f"Warning: '{csv_file}' not found. Skipping data load for '{table_name}'.")

    # Commit the changes to the database
    conn.commit()
    print("\nData committed to the database successfully. 🎉")

except sqlite3.Error as e:
    print(f"Database error: {e}")
except pd.errors.EmptyDataError as e:
    print(f"Pandas error: {e}. One of the CSV files might be empty.")
except KeyError as e:
    print(f"Schema definition error: A column is missing from the TABLE_DATA_TYPES dictionary: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    # Close the connection if it was established
    if conn:
        conn.close()
        print("Database connection closed.")

Database 'ecommerce.db' created and connected successfully. ✅
Tables 'employees' created successfully.

Processing '/content/Employee_data.csv' for table 'employees'...
  -> Data from '/content/Employee_data.csv' loaded into 'employees' table successfully.

Data committed to the database successfully. 🎉
Database connection closed.


In [ ]:
from google import genai
from google.colab import userdata
genai_client = genai.Client(api_key = userdata.get('GOOGLE_API_KEY'))

In [ ]:
prompt = """
### **ROLE**

You are an expert-level SQLite Database Engineer specializing in Natural Language to SQL (NL2SQL) translation. Your sole function is to convert user questions written in plain English into accurate, efficient, and syntactically correct SQLite queries based on a fixed database schema.

-----

### **CONTEXT**

You are the core translation engine for a business intelligence dashboard. This tool allows non-technical employees to query the company's employee database using natural language. The database dialect is always **SQLite**. Your responses will be executed directly on the database.

The database consist of the following table:

**`employee` table:**

```sql
create table MOCK_DATA (
	employee_id INT,
	first_name VARCHAR(50),
	last_name VARCHAR(50),
	age INT,
	email VARCHAR(50),
	gender VARCHAR(50),
	job_title VARCHAR(50),
	department VARCHAR(9),
	salary DECIMAL(8,2),
	hire_date DATE,
	manager_id INT,
	street_address VARCHAR(50),
	city VARCHAR(50),
	state VARCHAR(50),
	postal_code VARCHAR(50),
	country VARCHAR(50),
	phone_number VARCHAR(50),
	emergency_contact VARCHAR(50),
	emergency_contact_relationship VARCHAR(7),
	start_time VARCHAR(50),
	end_time VARCHAR(50)
  );

```

-----

### **TASK**

Your task is to receive a user's question in natural language and convert it into a single, executable SQLite query. Follow these steps meticulously:

1.  **Analyze the User's Query:** Deconstruct the user's question to understand their core intent. Identify the specific data, conditions, aggregations (like `SUM`, `COUNT`, `AVG`), and ordering they are asking for.
2.  **Map to the Schema:** Map the entities from the user's query to the appropriate tables (`customers`, `products`, `orders`) and columns. Determine the necessary `JOIN` operations using `customers.customer_id` and `products.product_id` as foreign keys in the `orders` table.
3.  **Construct the SQLite Query:** Write a clean and efficient `SELECT` statement that is syntactically correct for SQLite. Ensure all table and column names are accurate.
4.  **Handle Ambiguity:** If the user's query is vague, ambiguous, or lacks the necessary information to create a precise query, do not guess. Instead, formulate a specific, targeted question to ask the user for the missing information.

-----

### **CONSTRAINTS**

  * **Read-Only Operations:** You must **ONLY** generate `SELECT` queries. Never generate `INSERT`, `UPDATE`, `DELETE`, `DROP`, or any other data-modifying statements.
  * **Adhere Strictly to Schema:** Only use the tables and columns defined in the context. Do not invent or assume the existence of any other tables or columns.
  * **No Explanations:** Do not add any conversational text or explanations about the query you generate. Your output must strictly follow the specified format.
  * **Single Query Only:** The final output must be a single, complete, and executable SQL query.
  * **Handle Impossibility:** If a request is impossible to fulfill with the given schema (e.g., "Which employee made the most sales?"), state clearly that the request cannot be completed and briefly explain why.

-----

### **EXAMPLES**

**Example 1: Simple Lookup**

  * **User Query:** "Return all employees older than 40"
  * **Expected Output:**
    ```json
    {
      "status": "success",
      "response": "SELECT first_name, last_name, age, department FROM MOCK_DATA WHERE age > 40;"
    }
    ```

**Example 2: Complex Join and Aggregation**

  * **User Query:** "Find each employee’s manager name"
  * **Expected Output:**
    ```json
    {
      "status": "success",
      "response": "SELECT e.employee_id AS employee_id, e.first_name || ' ' || e.last_name AS employee_name,m.first_name || ' ' || m.last_name AS manager_name FROM MOCK_DATA e JOIN MOCK_DATA m ON e.manager_id = m.employee_id;"
    }
    ```

**Example 3: Ambiguous Query**

  * **User Query:** "Show the first_name and email of employees along with the email of their managers, but without qualifying which table the email comes from."
  * **Expected Output:**
    ```json
    {
      "status": "clarification_needed",
      "response": "This is ambiguous because both the employee (alias e) and the manager (alias m) have a column called email."
    }
    ```

**Example 4: Impossible Query**

  * **User Query:** "Find employees whose start_time is later than their end_time and salary is negative"
  * **Expected Output:**
    ```json
    {
      "status": "error",
      "response": "I cannot answer this question as the database does not contain information about warehouses."
    }
    ```

-----

### **OUTPUT FORMAT**

Your final response must be a single JSON object with two keys:

1.  `"status"`: A string with one of three possible values: `"success"`, `"clarification_needed"`, or `"error"`.
2.  `"response"`:
      * If `status` is `"success"`, this will be a string containing the complete SQLite query.
      * If `status` is `"clarification_needed"`, this will be a string containing the clarifying question for the user.
      * If `status` is `"error"`, this will be a string explaining why the query could not be generated.
"""

In [ ]:
import json
def get_sql_query(genai_client, prompt, user_query):
  contents = f"""
  {prompt}

  Here's the user query in english you need to work on:
  {user_query}
  """
  response = genai_client.models.generate_content(model='gemini-2.5-flash', contents=contents)
  # print(response)

  # Access the usage_metadata attribute
  usage_metadata = response.usage_metadata

  # Print the different token counts
  print(f"Input Token Count: {usage_metadata.prompt_token_count}")
  print(f"Thoughts Token Count: {response.usage_metadata.thoughts_token_count}")
  print(f"Output Token Count: {usage_metadata.candidates_token_count}")
  print(f"Total Token Count: {usage_metadata.total_token_count}")

  output = json.loads(response.text.replace('```json', '').replace('```', ''))

  return output

In [ ]:
import sqlite3
import pandas as pd
def execute_query(query, db_name='ecommerce.db'):
    conn = None
    try:
        # Connect to the database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()
        # Execute the query
        print(f"\nExecuting query on '{db_name}':\n{query}")
        cursor.execute(query)
        # Fetch all results
        results = cursor.fetchall()
        # Get column names from the cursor description
        columns = [description[0] for description in cursor.description]
        # Format results as a dataframe for easier use
        results_as_dict = [dict(zip(columns, row)) for row in results]
        results_df = pd.DataFrame(results_as_dict)
        print("Query executed successfully.")
        return results_df
    except sqlite3.Error as e:
        print(f"Database error executing query: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None
    finally:
        if conn:
            conn.close()

In [ ]:
def text2sql(genai_client, prompt, user_query):
  output = get_sql_query(genai_client, prompt, user_query)
  if output['status'] == 'success':
    results = execute_query(output['response'])
    return results
  return output

In [ ]:
text2sql(genai_client, prompt, "List all employees from the “HR” department earning more than 50,000.")

Input Token Count: 1367
Thoughts Token Count: 73
Output Token Count: 55
Total Token Count: 1495

Executing query on 'ecommerce.db':
SELECT first_name, last_name, department, salary FROM MOCK_DATA WHERE department = 'HR' AND salary > 50000;
Query executed successfully.


""


In [ ]:
text2sql(genai_client, prompt, "For each employee, show their name and their manager’s full name.")

Input Token Count: 1362
Thoughts Token Count: 193
Output Token Count: 87
Total Token Count: 1642

Executing query on 'ecommerce.db':
SELECT e.first_name || ' ' || e.last_name AS employee_name, m.first_name || ' ' || m.last_name AS manager_name FROM MOCK_DATA AS e JOIN MOCK_DATA AS m ON e.manager_id = m.employee_id;
Query executed successfully.


""


In [ ]:
text2sql(genai_client, prompt, "Find all employees who joined before 1st January 2020.")

Input Token Count: 1363
Thoughts Token Count: 94
Output Token Count: 46
Total Token Count: 1503

Executing query on 'ecommerce.db':
SELECT * FROM MOCK_DATA WHERE hire_date < '2020-01-01';
Query executed successfully.


""
